In [1]:
%pwd

'c:\\Users\\RETRETA\\Documents\\Python\\ML Upskill\\gen-ai-tuts\\langchain-gen-ai\\chain'

In [3]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "../rag/langchain-assist.pdf"
loader = PyPDFLoader(pdf_path)

docs = loader.load()
docs

[Document(page_content='                                                                                                               e-ISSN: 2582 -5208 \nInternatio nal  Research Journal  of  Modernization in Engineering Technology  and  Science  \n( Peer -Reviewed, Open Access, Fully Refereed International Journal )  \nVolume:05/Issue:07/July -2023                   Impact Factor - 7.868                                    www .irjmets.com                        \nwww.irjmets.com                               @International Research Journal of Modernization in Engineering, Technology and Science  \n [2796 ] \nLANGCHAIN -POWERED VIRTUAL ASSISTANT FOR PDF  \nCOMMUNICATION  \nNR Tejaswini*1, Vidya S*2, Dr. T Vijaya Kumar*3 \n*1Student , Master Of Computer Applications , Bangalore Institute Of Technology , Bangalore , India . \n*2Assistant  Professor , Master Of Computer Applications , Bangalore Institute Of  \nTechnology , Bangalore , India . \n*3Professor , Master Of Computer Applicati

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='e-ISSN: 2582 -5208 \nInternatio nal  Research Journal  of  Modernization in Engineering Technology  and  Science  \n( Peer -Reviewed, Open Access, Fully Refereed International Journal )  \nVolume:05/Issue:07/July -2023                   Impact Factor - 7.868                                    www .irjmets.com                        \nwww.irjmets.com                               @International Research Journal of Modernization in Engineering, Technology and Science  \n [2796 ] \nLANGCHAIN -POWERED VIRTUAL ASSISTANT FOR PDF  \nCOMMUNICATION  \nNR Tejaswini*1, Vidya S*2, Dr. T Vijaya Kumar*3 \n*1Student , Master Of Computer Applications , Bangalore Institute Of Technology , Bangalore , India . \n*2Assistant  Professor , Master Of Computer Applications , Bangalore Institute Of  \nTechnology , Bangalore , India .', metadata={'source': '../rag/langchain-assist.pdf', 'page': 0}),
 Document(page_content='*3Professor , Master Of Computer Applications , Bangalore Institu

In [5]:
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='e-ISSN: 2582 -5208 \nInternatio nal  Research Journal  of  Modernization in Engineering Technology  and  Science  \n( Peer -Reviewed, Open Access, Fully Refereed International Journal )  \nVolume:05/Issue:07/July -2023                   Impact Factor - 7.868                                    www .irjmets.com                        \nwww.irjmets.com                               @International Research Journal of Modernization in Engineering, Technology and Science  \n [2796 ] \nLANGCHAIN -POWERED VIRTUAL ASSISTANT FOR PDF  \nCOMMUNICATION  \nNR Tejaswini*1, Vidya S*2, Dr. T Vijaya Kumar*3 \n*1Student , Master Of Computer Applications , Bangalore Institute Of Technology , Bangalore , India . \n*2Assistant  Professor , Master Of Computer Applications , Bangalore Institute Of  \nTechnology , Bangalore , India .', metadata={'source': '../rag/langchain-assist.pdf', 'page': 0}),
 Document(page_content='*3Professor , Master Of Computer Applications , Bangalore Institu

In [8]:
from langchain_openai import OpenAIEmbeddings
# from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=documents[:30],
                          embedding=OpenAIEmbeddings()
)


In [10]:
db

In [9]:
query = "langchain pdf query"
result = db.similarity_search(query)
result[0].page_content

'*3Professor , Master Of Computer Applications , Bangalore Institute Of Technology , Bangalore , India . \nDOI : https://www.doi.org/10.56726/IRJMETS43587  \nABSTRACT  \nDue to the unstructured nature of the PDF document format and the requirement for precise and pertinent \nsearch results, querying a PDF can take time and effort. LangChain overcomes these challenges by utilizing \nadvanced natural language processing algorithms that analyze the content of the PDFs and extract essential \ninformation. To improve the search experience, it uses effective indexing and retrieval techniques, movable \nfilters  and a simple search int erface. LangChain also allows users to save queries, create bookmarks  and \nannotate important sections, enabling efficient retrieval of relevant information from PDF documents. The \nfeatures of LangChain increase overall efficiency and makes PDF querying muc h easier and simpler.  \nKeywords : LangChain, Querying PDF, Machine Translation, Streamlit.'

In [11]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")
llm 

Ollama(model='llama3')

In [13]:
# Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on the provided question.
    Think step by step before providing a detailed answer.
    <context>
    {context}
    <context>
    Question: {input}
    """
)
# context are the documents available in the vectorstore

In [16]:
## Create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/

In [17]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000253FAA3F760>)

**Retrieval chain**: This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response.


https://python.langchain.com/docs/modules/chains/

In [18]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
response = retrieval_chain.invoke(
    {"input": "what educational institution are the authors from?"}
)
response

{'input': 'what educational institution are the authors from?',
 'context': [Document(page_content='e-ISSN: 2582 -5208 \nInternatio nal  Research Journal  of  Modernization in Engineering Technology  and  Science  \n( Peer -Reviewed, Open Access, Fully Refereed International Journal )  \nVolume:05/Issue:07/July -2023                   Impact Factor - 7.868                                    www .irjmets.com                        \nwww.irjmets.com                               @International Research Journal of Modernization in Engineering, Technology and Science  \n [2796 ] \nLANGCHAIN -POWERED VIRTUAL ASSISTANT FOR PDF  \nCOMMUNICATION  \nNR Tejaswini*1, Vidya S*2, Dr. T Vijaya Kumar*3 \n*1Student , Master Of Computer Applications , Bangalore Institute Of Technology , Bangalore , India . \n*2Assistant  Professor , Master Of Computer Applications , Bangalore Institute Of  \nTechnology , Bangalore , India .', metadata={'source': '../rag/langchain-assist.pdf', 'page': 0}),
  Document(pa

In [40]:
from IPython.display import display, Markdown

display(Markdown(response['answer']))

Based on the provided context, I can answer your question as follows:

The authors NR Tejaswini, Vidya S, and Dr. T Vijaya Kumar are from Bangalore Institute Of Technology (BIT), Bangalore, India.